In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import scipy as sp
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
from itertools import islice
from IPython.display import clear_output
import pytry
import pickle

Generate and Save Input Data

multidata.csv was made by collating all of the child-child interactions from the PInSoRo data set into one csv and then removing the Yellow-Child columns.

In [2]:
target_dt = 1.0/30 #30 frames per second?

#load combined file of annotated child-child data from PInSoRo
data = pd.read_csv("..\\Data\\Input\\multidata.csv", low_memory=False) 
#array of data for purple child (points in space for each frame)
x = np.array(data.iloc[:,11:195]).astype(float) 
#array of labels (purple child annotations, engagement) 
labs = np.array(data.iloc[:,196]).astype(str) 
a = []
#for each column in the array, fill in any blanks by interpolating between available values
for i in range(x.shape[1]):
    y = pd.Series(x[:,i])
    if i == 180:               
        y[y>0]-=np.pi*2   
        y += np.pi            
    z = y.interpolate(limit_direction='both')
    a.append(z)
a = pd.DataFrame(a)
a = a.dropna()
a = np.array(a).T

#function to extract consecutive frames with the same label (i.e. clips)
def extract_pattern(start, end, target_dt): 
    pattern = np.array(a[start:end,:]).astype(float)
    frames = np.array(data.iloc[start:end,9]).astype(int)
    timestamp = np.array(data.iloc[start:end,0]).astype(str)

    good_indices = frames != -1
    frames = frames[good_indices]
    pattern = pattern[good_indices]

    fps = 30.0
    t_sample = (frames - frames[0])/fps

    #t = np.arange(int(t_sample[-1]/target_dt))*target_dt
    t = np.arange(end-start)*target_dt

    result = []
    for i in range(pattern.shape[1]):       
        p = np.interp(t, t_sample, pattern[:,i])
        result.append(p)
    result = np.array(result).T

    return timestamp, result

start=[]
start.append(0)
end=[]
label = []
for i in range(1, (len(labs)-1)):
    if labs[i]!=labs[i-1]:
        start.append(i)
    if labs[i]!=labs[i+1]:
        end.append(i)
        label.append(labs[i])

t_high=[]
p_high=[]
t_mid=[]
p_mid=[]
t_low=[]
p_low=[]

for i in range(1,(len(start)-1)):
    try:
        if label[i]==('goaloriented'):
            ts, pi = extract_pattern(start[i], end[i], target_dt=target_dt)
            t_high.append(ts)
            p_high.append(pi)
        if label[i]==('aimless'):
            ts, pi = extract_pattern(start[i], end[i], target_dt=target_dt)
            t_mid.append(ts)
            p_mid.append(pi)
        if label[i]==('noplay'):
            ts, pi = extract_pattern(start[i], end[i], target_dt=target_dt)
            t_low.append(ts)
            p_low.append(pi)
    except IndexError:
        print('empty pattern')
        
i=0
high = []
mid = []
low = []
for entry in t_high:
    high.append([t_high[i],p_high[i]])
    i = i+1
i=0
for entry in t_mid:
    mid.append([t_mid[i],p_mid[i]])
    i = i+1
i=0
for entry in t_low:
    low.append([t_low[i],p_low[i]])
    i = i+1
    
np.save('higheng.npy', high)
np.save('mideng.npy', mid)
np.save('loweng.npy', low)